# AUTO MACHINE LEARNING
### Using it for predicting Case Fatality Ratio

In [2]:
import numpy as np
import pandas as pd

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
df = pd.read_excel('train_data.xlsx')
df.head()

,Lat,Long_,Deaths,Case_Fatality_Ratio
0,33.93911,67.709953,NaN,3.779217
1,41.15330,20.168300,NaN,1.077234
2,28.03390,1.659600,NaN,2.536905
3,42.50630,1.521800,165.0,0.345543
4,-11.20270,17.873900,NaN,1.836434


In [5]:
hf = df[['Lat', 'Long_']].dropna()
hf['Case_Fatality_Ratio'] = df['Case_Fatality_Ratio']

In [6]:
hf = hf.reset_index()

In [7]:
hf.drop(columns = ['index'], inplace=True)

In [8]:
hf


,Lat,Long_,Case_Fatality_Ratio
0,33.939110,67.709953,3.779217
1,41.153300,20.168300,1.077234
2,28.033900,1.659600,2.536905
3,42.506300,1.521800,0.345543
4,-11.202700,17.873900,1.836434
...,...,...,...
3920,31.952200,35.233200,0.811686
3921,39.904200,116.407400,0.000000
3922,15.552727,48.516388,18.074508
3923,-13.133897,27.849332,1.201603


In [9]:
hf['Case_Fatality_Ratio'].isna().sum()

3

In [10]:
hf[hf['Case_Fatality_Ratio'].isna()]

,Lat,Long_,Case_Fatality_Ratio
731,58.745140,-156.701064,NaN
753,61.475028,-144.712680,NaN
3901,49.372300,-2.364400,NaN


In [11]:
hf = hf.drop([731,753,3901])
hf = hf.reset_index()

In [12]:
x_train = hf.iloc[:2700,:]
x_train

,index,Lat,Long_,Case_Fatality_Ratio
0,0,33.939110,67.709953,3.779217
1,1,41.153300,20.168300,1.077234
2,2,28.033900,1.659600,2.536905
3,3,42.506300,1.521800,0.345543
4,4,-11.202700,17.873900,1.836434
...,...,...,...,...
2695,2697,47.456115,-97.727287,0.468384
2696,2698,46.979625,-98.956373,1.199308
2697,2699,48.685657,-99.245643,1.632653
2698,2700,47.453678,-97.163233,1.064701


In [13]:
x_test = hf.iloc[2700:,:]
x_test

,index,Lat,Long_,Case_Fatality_Ratio
2700,2702,48.223902,-101.543676,1.063379
2701,2703,47.586145,-99.659050,1.016088
2702,2704,48.345224,-103.479339,0.565771
2703,2705,15.097900,145.673900,0.307831
2704,2706,38.845411,-83.471896,1.866018
...,...,...,...,...
3917,3920,31.952200,35.233200,0.811686
3918,3921,39.904200,116.407400,0.000000
3919,3922,15.552727,48.516388,18.074508
3920,3923,-13.133897,27.849332,1.201603


In [14]:
train_data = TabularDataset(x_train)

In [15]:
predictor = TabularPredictor(label='Case_Fatality_Ratio').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20250131_063239"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Fri Jul  5 17:56:39 PDT 2024; root:xnu-10063.141.1~2/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       1.79 GB / 8.00 GB (22.4%)
Disk Space Avail:   162.18 GB / 228.27 GB (71.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy.

In [16]:
test_data = TabularDataset(x_test)
y_pred = predictor.predict(test_data.drop(columns=['Case_Fatality_Ratio']))

predictor.evaluate(test_data, silent=True)

## Leaderboard of all the models

In [17]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,-0.891143,-9.836569,root_mean_squared_error,0.047334,0.025511,0.523256,0.047334,0.025511,0.523256,1,True,5
1,XGBoost,-0.904102,-6.077938,root_mean_squared_error,0.005511,0.000914,0.394246,0.005511,0.000914,0.394246,1,True,9
2,LightGBM,-0.913107,-6.084158,root_mean_squared_error,0.000480,0.000291,0.252856,0.000480,0.000291,0.252856,1,True,4
3,LightGBMLarge,-0.916024,-6.089473,root_mean_squared_error,0.000454,0.000326,0.923897,0.000454,0.000326,0.923897,1,True,11
4,LightGBMXT,-0.917283,-6.081471,root_mean_squared_error,0.000669,0.000308,1.436690,0.000669,0.000308,1.436690,1,True,3
5,CatBoost,-0.917549,-6.079578,root_mean_squared_error,0.009385,0.000443,1.090942,0.009385,0.000443,1.090942,1,True,6
6,KNeighborsDist,-0.936887,-6.096060,root_mean_squared_error,0.014939,0.013816,0.002461,0.014939,0.013816,0.002461,1,True,2
7,KNeighborsUnif,-0.936942,-6.098365,root_mean_squared_error,0.015807,0.022353,2.573747,0.015807,0.022353,2.573747,1,True,1
8,NeuralNetFastAI,-0.953444,-6.134342,root_mean_squared_error,0.018484,0.003502,1.566912,0.018484,0.003502,1.566912,1,True,8
9,WeightedEnsemble_L2,-0.997984,-6.053830,root_mean_squared_error,0.015321,0.003155,7.886213,0.001343,0.000125,0.003368,2,True,12


# Prediction of Deaths and KNN Imputation

In [18]:
df

,Lat,Long_,Deaths,Case_Fatality_Ratio
0,33.939110,67.709953,NaN,3.779217
1,41.153300,20.168300,NaN,1.077234
2,28.033900,1.659600,NaN,2.536905
3,42.506300,1.521800,165.0,0.345543
4,-11.202700,17.873900,NaN,1.836434
...,...,...,...,...
4011,31.952200,35.233200,NaN,0.811686
4012,39.904200,116.407400,0.0,0.000000
4013,15.552727,48.516388,NaN,18.074508
4014,-13.133897,27.849332,NaN,1.201603


In [19]:
from sklearn.impute import KNNImputer

In [20]:
imputer = KNNImputer(n_neighbors=5)

In [21]:
df['Deaths'] = imputer.fit_transform(df[['Deaths']])

In [22]:
df

,Lat,Long_,Deaths,Case_Fatality_Ratio
0,33.939110,67.709953,70.753051,3.779217
1,41.153300,20.168300,70.753051,1.077234
2,28.033900,1.659600,70.753051,2.536905
3,42.506300,1.521800,165.000000,0.345543
4,-11.202700,17.873900,70.753051,1.836434
...,...,...,...,...
4011,31.952200,35.233200,70.753051,0.811686
4012,39.904200,116.407400,0.000000,0.000000
4013,15.552727,48.516388,70.753051,18.074508
4014,-13.133897,27.849332,70.753051,1.201603


In [23]:
df = df.drop(df[df['Lat'].isna()].index)

In [24]:
df[df['Case_Fatality_Ratio'].isna()]

,Lat,Long_,Deaths,Case_Fatality_Ratio
751,58.745140,-156.701064,0.0,NaN
774,61.475028,-144.712680,0.0,NaN
3991,49.372300,-2.364400,0.0,NaN


In [25]:
df = df.drop(df[df['Case_Fatality_Ratio'].isna()].index)

In [26]:
df = df.reset_index()

In [27]:
train_data = df.iloc[:2700,:]
test_data = df.iloc[2700:,:]

In [30]:
train_data = TabularDataset(train_data)


In [31]:
predictor2 = TabularPredictor(label='Deaths').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20250131_063450"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Fri Jul  5 17:56:39 PDT 2024; root:xnu-10063.141.1~2/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       1.79 GB / 8.00 GB (22.3%)
Disk Space Avail:   161.99 GB / 228.27 GB (71.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy.

In [32]:
test_data = TabularDataset(test_data)
y_pred = predictor.predict(test_data)

In [33]:
predictor2.evaluate(test_data, silent=True)

{'root_mean_squared_error': -42.27297264565361,
 'mean_squared_error': -1787.004216300178,
 'mean_absolute_error': -30.341344962504785,
 'r2': 0.16665886156245757,
 'pearsonr': 0.42284305348478,
 'median_absolute_error': -21.179455757141113}

In [34]:
predictor2.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,-40.754857,-36.037568,root_mean_squared_error,0.003130,0.000563,0.359101,0.003130,0.000563,0.359101,1,True,4
1,XGBoost,-41.058778,-36.263612,root_mean_squared_error,0.006449,0.000931,0.286044,0.006449,0.000931,0.286044,1,True,9
2,RandomForestMSE,-41.258278,-36.744076,root_mean_squared_error,0.046061,0.013809,0.338787,0.046061,0.013809,0.338787,1,True,5
3,LightGBMLarge,-41.470756,-36.720346,root_mean_squared_error,0.002778,0.000836,0.774279,0.002778,0.000836,0.774279,1,True,11
4,CatBoost,-41.827854,-35.589145,root_mean_squared_error,0.006110,0.000400,0.217179,0.006110,0.000400,0.217179,1,True,6
5,LightGBMXT,-41.998192,-35.267042,root_mean_squared_error,0.005410,0.002204,0.492737,0.005410,0.002204,0.492737,1,True,3
6,WeightedEnsemble_L2,-42.272973,-35.128445,root_mean_squared_error,0.065718,0.021022,12.038425,0.000732,0.000112,0.003217,2,True,12
7,NeuralNetTorch,-42.420204,-37.044189,root_mean_squared_error,0.003840,0.002475,10.788983,0.003840,0.002475,10.788983,1,True,10
8,ExtraTreesMSE,-43.414845,-35.869273,root_mean_squared_error,0.042556,0.013761,0.152634,0.042556,0.013761,0.152634,1,True,7
9,NeuralNetFastAI,-54.173545,-37.862596,root_mean_squared_error,0.013180,0.002470,0.600854,0.013180,0.002470,0.600854,1,True,8


### Initializing the test data

In [35]:
pf=pd.read_excel('test_points_12.xlsx')

### Predicting CFR data

In [36]:
test_data_1 = TabularDataset(pf)
y_pred_cfr = predictor.predict(test_data_1)

### Adding cfr column to the dataframe

In [37]:
td_2=test_data_1.copy()
td_2['Case_Fatality_Ratio']=y_pred_cfr

In [38]:
td_2

,index,Lat,Long_,Deaths,Case_Fatality_Ratio
0,NaN,41.153300,20.168300,NaN,0.731676
1,NaN,-71.949900,23.347000,NaN,0.509292
2,NaN,-35.473500,149.012400,NaN,0.513986
3,NaN,-34.928500,138.600700,NaN,0.592690
4,NaN,47.516200,14.550100,NaN,0.704449
...,...,...,...,...,...
999,NaN,49.213800,-2.135800,NaN,0.678653
1000,NaN,-7.946700,-14.355900,NaN,1.945333
1001,NaN,52.130700,-3.783700,NaN,0.683480
1002,NaN,6.423800,-66.589700,NaN,1.822923


In [39]:
td_2=TabularDataset(td_2)
td_2['Deaths']=predictor2.predict(td_2)

In [40]:
td_2.to_csv('automl_predictions.csv')

### The best Models

In [41]:
cfr_model = predictor.model_best
print("Best CFR Model:", cfr_model)


Best CFR Model: WeightedEnsemble_L2


In [42]:
death_model = predictor2.model_best
print("Best Death Model:", death_model)


Best Death Model: WeightedEnsemble_L2
